## Intro

In day 0 of my journey of trying to understand what people are talking in regards to agents, I am going to deep dive into OpenAI's new Agents SDK. 

My goals are to:
1. Understand the core mental model of OpenAI's dev team in regards to agents
2. Run the basic examples in a notebook
3. Read the API docs
4. Generate new research questions

## Dependencies

This notebook assumes you have started the kernel associated with the `agent-100daze` conda environment. 
You can create this environment with:
```bash
conda env create -f env.yaml
```

There is a `pyproject.toml` for `uv`. If you use uv, I'll assume you are enough of a competent nerd to figure it out your own.

### API Keys

Get [your OpenAI API key](https://platform.openai.com/settings/organization/api-keys), put it in `.env`. After you run the next cell the SDK's will see it.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

### Other dependencies

In [2]:
from agents import (
    ### Agent configuration
    Agent, 
    AgentHooks,
    AgentOutputSchema,
    ### Guardrails are configured on Agent definition.
    InputGuardrail,
    InputGuardrailResult,
    InputGuardrailTripwireTriggered,
    OutputGuardrail,
    OutputGuardrailResult,
    OutputGuardrailTripwireTriggered,
    ### Runtime manager for agent invocation
    Runner,
    RunConfig,
    RunHooks,
    RunResult,
    ### Logging and visualization
    trace
)

import inspect
from IPython.display import display, Markdown

#### Utility functions

In [3]:
def display_signature(obj):
    s = inspect.signature(obj)
    name = getattr(obj, '__name__', str(obj))
    _str = f"### {name}\n\n"
    for param_name, param in s.parameters.items():
        default = f" = {param.default}" if param.default != inspect.Parameter.empty else " *"
        annotation = f": {param.annotation}" if param.annotation != inspect.Parameter.empty else ""
        _str += f"&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**{param_name}**{annotation}{default}\n\n"

    display(Markdown(_str))

`<side-quest>`

OpenAI Agent's SDK depends on `asyncio`. This makes it not play nice with Jupyter notebooks.

The Jupyter server depends on [Tornado web server](https://www.tornadoweb.org/en/stable/). 
Many moons ago, an upgrade to Jupyter's Tornado 5.0 bricked asyncio in notebooks.
Read more [here](https://github.com/jupyter/notebook/issues/3397), and [here](https://stackoverflow.com/questions/47518874/how-do-i-run-python-asyncio-code-in-a-jupyter-notebook).
If you run this in a notebook:
```python
import asyncio
asyncio.get_event_loop()
```
you'll get:
```bash
<_UnixSelectorEventLoop running=True closed=False debug=False>
```
but if you run the same piece of code in a Python repl:
```bash
<_UnixSelectorEventLoop running=False closed=False debug=False>
```
What this means is that we cannot run `asyncio` event loops without special mods, because Jupyter is already in one! 

#### Practical conclusion

You cannot do `Runner.run_sync` or some other APIs in a notebook. 
Instead, do `await Runner.run(agent, ...)`.

`</side-quest>`

## Getting started

### Hello world

Copy-pasted example from OpenAI docs, validating set up. 

In [ ]:
agent = Agent(name="Assistant", instructions="You are a helpful assistant")
result = await Runner.run(agent, "Write a haiku about recursion in programming.") # type: ignore[top-level-await]  # noqa: F704 
display(Markdown(result.final_output))

### Notes on [the examples](https://github.com/openai/openai-agents-python/tree/main/examples)

#### 💰 Business
- Finance research
- Customer service

#### 🤓 Research
- Model flexiiblity
    - [Custom LLM providers](https://github.com/openai/openai-agents-python/tree/main/examples/model_providers)
- Agent design patterns
    - **Deterministic agent workflows**: Traditional ML DAGs where each node is an agent call.
    - **Sub-agents**: This is what OpenAI refers to as "handoffs". Routing tasks among agent pools.
    - **Agents as tools**: Distinct from the sub-agent handoff, agents can call other " agents as tools". The difference is which agent is the main process of the program. The mental model I have is akin to the difference between a main program stopping and starting a new main program vs. the main program calling a subprocess.
    - **LLM as a judge**: One LLM does some task, another LLM scores it. Sensible approach to cost optimization and specialization. 
    - **Parallelization**: What it sounds like. Generate N samples, do some averaging or select the best. The design space with this combined with parallelizing over the runtime that contains these processes is quite interesting to me, stay tuned!
    - **Guardrails**: Validate the inputs and outputs. Fail fast when validation breaks. Given a common use case scenario for Agents, what ways can I/O validation fail? 
- Tools
    - File search
    - Web search
    - Image generator
    - Code interpreter :
    - Voice: talk to the agent runtime without touching the computer. 
- Tools via model context protocol (MCP) integration
    - [Filesystem](https://github.com/modelcontextprotocol/servers/tree/main/src/filesystem) exposes a bunch of [tools](https://github.com/modelcontextprotocol/servers/tree/main/src/filesystem#tools). 
    - Git
    - SSE
    - Any MCP server, theoretically

## What is an Agent?

> LLMs configured with instructions with access to tools. 

#### What are instructions?
System prompts.

#### What does "access to tools" mean?
The LLM is going to run within a loop, 
and each iteration can stop the LLM generation, 
output some tokens that cause the loop to run another Python function (which may or may not call another Agent),
and then continue iterating until the loop terminates. 

In [ ]:
display_signature(Agent)

In [ ]:
display_signature(Runner.run)

In [ ]:
display_signature(RunResult)

## Monitoring

### [Traces and Spans](https://openai.github.io/openai-agents-python/tracing/)

A way to see what Agents are doing. 
OpenAI has a dashboard to visualize the traces.

In [ ]:
agent = Agent(name="Joke generator", instructions="Tell funny jokes.")

run_name = "Joke workflow"
joker_prompt = "Tell me a joke"
king_prompt = "Respond only with 👍 or 👎. Rate this joke: {joke}"

with trace(run_name):
    first_result = await Runner.run(agent, joker_prompt)
    second_result = await Runner.run(agent, king_prompt.format(joke=first_result.final_output))
        
display(Markdown(first_result.final_output))
display(Markdown(second_result.final_output))

#### Trace visualization

At `https://platform.openai.com/traces/trace?trace_id=<>`, I can now see a beautiful monitoring dashboard.  

![](../static/openai-platform-trace-demo.png)

### Trace customization

[`BatchTraceProcessor`](https://openai.github.io/openai-agents-python/ref/tracing/processors/#agents.tracing.processors.BatchTraceProcessor) defines a [`BackendSpanExporter`](https://openai.github.io/openai-agents-python/ref/tracing/processors/#agents.tracing.processors.BackendSpanExporter) instantiated with a [`TraceProvider`](https://openai.github.io/openai-agents-python/ref/tracing/setup/#agents.tracing.setup.TraceProvider). 

The way to register a custom trace processor is `from agents import add_trace_processor, set_trace_processors`.

#### Example implementations
- [MLFlow](https://github.com/mlflow/mlflow/blob/master/mlflow/openai/_agent_tracer.py)
- [Wandb weave](http://github.com/wandb/weave/blob/master/weave/integrations/openai_agents/openai_agents.py#L58)

There are a bunch more [here](https://openai.github.io/openai-agents-python/tracing/#external-tracing-processors-list).
Useful examples to show how to get the data and do something with it.

Let's implement the a basic processor based on the default `BatchTraceProcessor` and a custom exporter, which will make a local DB we can query. 

In [9]:
import time
import json
from typing import Any
import sqlite3
from pathlib import Path
from agents.tracing.traces import Trace
from agents.tracing.spans import Span
from agents.tracing.processor_interface import TracingExporter
from agents.tracing.processors import BatchTraceProcessor

class CustomTracingExporter(TracingExporter):
    """Dumps the OpenAI Agent SDK traces to a DB."""

    def __init__(self, db_path: str = "tracing.db"):
        self.db_path = Path(db_path)
        self._init_database()

    def _init_database(self):
        """Initialize the SQLite database with required tables."""
        with sqlite3.connect(self.db_path) as conn:
            conn.executescript("""
                CREATE TABLE IF NOT EXISTS traces (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    trace_id TEXT NOT NULL,
                    name TEXT,
                    timestamp REAL NOT NULL,
                    data TEXT  -- JSON blob of full exported data
                );

                CREATE TABLE IF NOT EXISTS spans (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    trace_id TEXT,  -- to link back to trace if available
                    timestamp REAL NOT NULL,
                    data TEXT  -- JSON blob of exported span data
                );

                -- Indexes for efficient querying
                CREATE INDEX IF NOT EXISTS idx_traces_trace_id ON traces(trace_id);
                CREATE INDEX IF NOT EXISTS idx_spans_trace_id ON spans(trace_id);
                CREATE INDEX IF NOT EXISTS idx_traces_timestamp ON traces(timestamp);
                CREATE INDEX IF NOT EXISTS idx_spans_timestamp ON spans(timestamp);
                CREATE INDEX IF NOT EXISTS idx_traces_name ON traces(name);
            """)

    def export(self, items: list[Trace | Span[Any]]) -> None:
        """Export traces and spans to SQLite database."""
        if not items:
            return

        current_time = time.time()
        
        with sqlite3.connect(self.db_path) as conn:
            for item in items:
                if isinstance(item, Trace):
                    self._store_trace(conn, item, current_time)
                else:
                    self._store_span(conn, item, current_time)

    def _store_trace(self, conn: sqlite3.Connection, trace: Trace, timestamp: float):
        """Store a trace in the database."""
        exported_data = trace.export() if hasattr(trace, 'export') else {}
        
        conn.execute("""
            INSERT INTO traces (trace_id, name, timestamp, data)
            VALUES (?, ?, ?, ?)
        """, (
            trace.trace_id,
            trace.name,
            timestamp,
            json.dumps(exported_data)
        ))
        print(f"Trace {trace.trace_id} successfully exported.")

    def _store_span(self, conn: sqlite3.Connection, span: Span[Any], timestamp: float):
        """Store a span in the database."""
        exported_data = span.export()
        
        # Try to extract trace_id if available in the span data
        trace_id = None
        if isinstance(exported_data, dict):
            trace_id = exported_data.get('trace_id')
        
        conn.execute("""
            INSERT INTO spans (trace_id, timestamp, data)
            VALUES (?, ?, ?)
        """, (
            trace_id,
            timestamp,
            json.dumps(exported_data)
        ))

    def query_traces(self, trace_id: str = None, name: str = None, limit: int = 100):
        """Query traces from the database."""
        with sqlite3.connect(self.db_path) as conn:
            conn.row_factory = sqlite3.Row  # Enable dict-like access
            
            query = "SELECT * FROM traces"
            params = []
            conditions = []
            
            if trace_id:
                conditions.append("trace_id = ?")
                params.append(trace_id)
            
            if name:
                conditions.append("name LIKE ?")
                params.append(f"%{name}%")
            
            if conditions:
                query += " WHERE " + " AND ".join(conditions)
            
            query += " ORDER BY timestamp DESC LIMIT ?"
            params.append(limit)
            
            return [dict(row) for row in conn.execute(query, params)]

    def query_spans(self, trace_id: str = None, limit: int = 100):
        """Query spans from the database."""
        with sqlite3.connect(self.db_path) as conn:
            conn.row_factory = sqlite3.Row
            
            query = "SELECT * FROM spans"
            params = []
            
            if trace_id:
                query += " WHERE trace_id = ?"
                params.append(trace_id)
            
            query += " ORDER BY timestamp DESC LIMIT ?"
            params.append(limit)
            
            return [dict(row) for row in conn.execute(query, params)]

    def get_trace_with_spans(self, trace_id: str):
        """Get a trace and all its associated spans."""
        traces = self.query_traces(trace_id=trace_id, limit=1)
        spans = self.query_spans(trace_id=trace_id, limit=1000)
        
        return {
            'trace': traces[0] if traces else None,
            'spans': spans
        }

    def stats(self):
        """Get basic statistics about stored data."""
        with sqlite3.connect(self.db_path) as conn:
            trace_count = conn.execute("SELECT COUNT(*) FROM traces").fetchone()[0]
            span_count = conn.execute("SELECT COUNT(*) FROM spans").fetchone()[0]
            earliest = conn.execute("""
                SELECT MIN(timestamp) FROM (
                    SELECT timestamp FROM traces 
                    UNION ALL 
                    SELECT timestamp FROM spans
                )
            """).fetchone()[0]
            latest = conn.execute("""
                SELECT MAX(timestamp) FROM (
                    SELECT timestamp FROM traces 
                    UNION ALL 
                    SELECT timestamp FROM spans
                )
            """).fetchone()[0]
            return {
                'traces': trace_count,
                'spans': span_count,
                'earliest': earliest,
                'latest': latest,
                'db_path': str(self.db_path)
            }

In [ ]:
DB_PATH = 'joke_agent_traces.db'

sqlite_tracing_exporter = CustomTracingExporter(db_path = DB_PATH)
batch_trace_processor_sqlite = BatchTraceProcessor(exporter=sqlite_tracing_exporter)

In [11]:
from agents import set_trace_processors
set_trace_processors([batch_trace_processor_sqlite])

In [ ]:
with trace(run_name):
    first_result = await Runner.run(agent, joker_prompt)
    second_result = await Runner.run(agent, king_prompt.format(joke=first_result.final_output))
        
display(Markdown(first_result.final_output))
display(Markdown(second_result.final_output))

In [ ]:
sqlite_tracing_exporter.stats()

In [ ]:
traces = sqlite_tracing_exporter.query_traces()
trace_0 = traces[0]

print(f"{trace_0['name']} (id={trace_0['trace_id']}) ran at {trace_0['timestamp']} and produced:")
print(trace_0['data'])

In [ ]:
spans = sqlite_tracing_exporter.query_spans()
span_0 = spans[0]
span_0_data = json.loads(span_0['data'])
print(f"span_id={span_0['id']} from trace_id={span_0['trace_id']}")
span_0_data

### Asking Claude to generate a UI to visualize traces

In [16]:
import json
from datetime import datetime
from IPython.display import display, HTML
from typing import Dict, List, Any

def show_span(span_row: Dict, show_raw: bool = False) -> None:
    """Display a span in a nice format for Jupyter notebooks."""
    
    # Parse the JSON data
    data = json.loads(span_row['data']) if isinstance(span_row['data'], str) else span_row['data']
    
    # Extract key info
    span_id = data.get('id', 'N/A')
    trace_id = data.get('trace_id', span_row.get('trace_id', 'N/A'))
    name = data.get('span_data', {}).get('name', 'Unnamed')
    span_type = data.get('span_data', {}).get('type', 'unknown')
    
    # Parse timestamps
    started = data.get('started_at', '')
    ended = data.get('ended_at', '')
    duration = None
    
    if started and ended:
        try:
            start_dt = datetime.fromisoformat(started.replace('Z', '+00:00'))
            end_dt = datetime.fromisoformat(ended.replace('Z', '+00:00'))
            duration = (end_dt - start_dt).total_seconds()
        except:
            pass
    
    # Error status
    error = data.get('error')
    error_color = "#ff5252" if error else "#4caf50"
    status = "❌ Error" if error else "✅ Success"
    
    # Build the HTML
    html = f"""
    <div style="border: 2px solid {error_color}; border-radius: 8px; padding: 15px; margin: 10px 0; background: #f8f9fa;">
        <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px;">
            <h4 style="margin: 0; color: #333;">🔍 {name}</h4>
            <span style="background: {error_color}; color: white; padding: 4px 8px; border-radius: 4px; font-size: 0.9em;">{status}</span>
        </div>
        
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px; margin-bottom: 10px;">
            <div><strong>Type:</strong> <code>{span_type}</code></div>
            <div><strong>Duration:</strong> {f'{duration:.3f}s' if duration else 'N/A'}</div>
        </div>
        
        <div style="font-size: 0.9em; color: #666; margin-bottom: 10px;">
            <div><strong>Span ID:</strong> <code style="font-size: 0.8em;">{span_id}</code></div>
            <div><strong>Trace ID:</strong> <code style="font-size: 0.8em;">{trace_id}</code></div>
        </div>
        
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px; font-size: 0.85em; color: #555;">
            <div><strong>Started:</strong> {started.split('T')[1][:8] if 'T' in started else started}</div>
            <div><strong>Ended:</strong> {ended.split('T')[1][:8] if 'T' in ended else ended}</div>
        </div>
    """
    
    # Add error details if present
    if error:
        html += f"""
        <div style="margin-top: 10px; padding: 8px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px;">
            <strong>Error:</strong> <code>{error}</code>
        </div>
        """
    
    # Add tools/handoffs if present
    span_data = data.get('span_data', {})
    tools = span_data.get('tools', [])
    handoffs = span_data.get('handoffs', [])
    
    if tools or handoffs:
        html += "<div style='margin-top: 10px; padding: 8px; background: #e3f2fd; border-radius: 4px;'>"
        if tools:
            html += f"<div><strong>Tools:</strong> {', '.join(tools) if tools else 'None'}</div>"
        if handoffs:
            html += f"<div><strong>Handoffs:</strong> {', '.join(handoffs) if handoffs else 'None'}</div>"
        html += "</div>"
    
    html += "</div>"
    
    display(HTML(html))
    
    # Show raw data if requested
    if show_raw:
        print("📋 Raw data:")
        print(json.dumps(data, indent=2))

def show_trace(trace_row: Dict, show_raw: bool = False) -> None:
    """Display a trace in a nice format for Jupyter notebooks."""
    
    # Parse the JSON data
    data = json.loads(trace_row['data']) if isinstance(trace_row['data'], str) else trace_row['data']
    
    trace_id = trace_row.get('trace_id', 'N/A')
    name = trace_row.get('name', 'Unnamed Trace')
    timestamp = trace_row.get('timestamp', 'N/A')
    
    # Convert timestamp to readable format
    readable_time = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S') if isinstance(timestamp, (int, float)) else timestamp
    
    html = f"""
    <div style="border: 2px solid #2196f3; border-radius: 8px; padding: 15px; margin: 10px 0; background: linear-gradient(135deg, #e3f2fd 0%, #f8f9fa 100%);">
        <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px;">
            <h4 style="margin: 0; color: #1565c0;">🔗 {name}</h4>
            <span style="background: #2196f3; color: white; padding: 4px 8px; border-radius: 4px; font-size: 0.9em;">TRACE</span>
        </div>
        
        <div style="font-size: 0.9em; color: #666; margin-bottom: 10px;">
            <div><strong>Trace ID:</strong> <code style="font-size: 0.8em;">{trace_id}</code></div>
            <div><strong>Timestamp:</strong> {readable_time}</div>
        </div>
    </div>
    """
    
    display(HTML(html))
    
    if show_raw:
        print("📋 Raw data:")
        print(json.dumps(data, indent=2))


def show_recent_activity(exporter, limit: int = 5) -> None:
    """Show recent traces and spans activity."""
    
    traces = exporter.query_traces(limit=limit)
    spans = exporter.query_spans(limit=limit)
    
    print(f"🕐 Recent Activity (last {limit} items)")
    print("=" * 50)
    
    if traces:
        print(f"\n📚 Recent Traces:")
        for trace in traces:
            show_trace(trace)
    
    if spans:
        print(f"\n📝 Recent Spans:")  
        for span in spans:
            show_span(span)

def show_trace_waterfall(exporter, trace_id: str) -> None:
    """Display a trace in traditional APM waterfall format."""
    
    trace_data = exporter.get_trace_with_spans(trace_id)
    trace = trace_data.get('trace')
    spans = trace_data.get('spans', [])
    
    if not trace:
        print(f"❌ No trace found with ID: {trace_id}")
        return
    
    # Parse trace info
    trace_name = trace.get('name', 'Unnamed Trace')
    
    # Parse and sort spans by start time
    parsed_spans = []
    for span in spans:
        span_data = json.loads(span['data'])
        start_time = span_data.get('started_at', '')
        end_time = span_data.get('ended_at', '')
        
        if start_time and end_time:
            try:
                start_dt = datetime.fromisoformat(start_time.replace('Z', '+00:00'))
                end_dt = datetime.fromisoformat(end_time.replace('Z', '+00:00'))
                duration_ms = (end_dt - start_dt).total_seconds() * 1000
                
                parsed_spans.append({
                    'data': span_data,
                    'start_dt': start_dt,
                    'end_dt': end_dt,
                    'duration_ms': duration_ms,
                    'name': span_data.get('span_data', {}).get('name', 'Unnamed'),
                    'type': span_data.get('span_data', {}).get('type', 'unknown'),
                    'error': span_data.get('error'),
                    'span_id': span_data.get('id', 'N/A')
                })
            except:
                continue
    
    if not parsed_spans:
        print("❌ No valid spans with timing data found")
        return
    
    # Sort by start time
    parsed_spans.sort(key=lambda x: x['start_dt'])
    
    # Calculate trace boundaries
    trace_start = min(s['start_dt'] for s in parsed_spans)
    trace_end = max(s['end_dt'] for s in parsed_spans)
    total_duration_ms = (trace_end - trace_start).total_seconds() * 1000
    
    # Build the waterfall HTML
    html = f"""
    <div style="font-family: 'Monaco', 'Menlo', monospace; background: #1e1e1e; color: #d4d4d4; padding: 20px; border-radius: 8px; margin: 10px 0;">
        <!-- Header -->
        <div style="border-bottom: 2px solid #404040; padding-bottom: 15px; margin-bottom: 20px;">
            <h3 style="margin: 0; color: #4fc3f7;">🔍 Trace: {trace_name}</h3>
            <div style="margin-top: 8px; font-size: 0.9em; color: #999;">
                <span style="margin-right: 20px;">📊 {len(parsed_spans)} spans</span>
                <span style="margin-right: 20px;">⏱️ {total_duration_ms:.1f}ms total</span>
                <span>🆔 {trace_id[:16]}...</span>
            </div>
        </div>
        
        <!-- Timeline header -->
        <div style="display: grid; grid-template-columns: 300px 1fr 100px; gap: 10px; padding: 8px 0; border-bottom: 1px solid #404040; font-size: 0.85em; color: #888; font-weight: bold;">
            <div>SERVICE / OPERATION</div>
            <div style="text-align: center;">TIMELINE</div>
            <div style="text-align: right;">DURATION</div>
        </div>
    """
    
    # Generate spans
    for i, span in enumerate(parsed_spans):
        # Calculate position and width for timeline bar
        start_offset_ms = (span['start_dt'] - trace_start).total_seconds() * 1000
        left_percent = (start_offset_ms / total_duration_ms) * 100
        width_percent = (span['duration_ms'] / total_duration_ms) * 100
        
        # Color coding
        if span['error']:
            bar_color = "#f44336"  # Red for errors
            bg_color = "#2d1b1b"
        elif span['type'] == 'agent':
            bar_color = "#4caf50"  # Green for agents
            bg_color = "#1b2d1b"
        elif span['type'] == 'tool':
            bar_color = "#ff9800"  # Orange for tools
            bg_color = "#2d251b"
        else:
            bar_color = "#2196f3"  # Blue for others
            bg_color = "#1b252d"
        
        # Service name and operation
        service_op = f"{span['type'].upper()} / {span['name']}"
        
        # Status indicator
        status_icon = "❌" if span['error'] else "✅"
        
        html += f"""
        <div style="display: grid; grid-template-columns: 300px 1fr 100px; gap: 10px; padding: 4px 0; border-bottom: 1px solid #333; background: {bg_color}; margin: 1px 0;">
            <!-- Service/Operation column -->
            <div style="padding: 8px; font-size: 0.9em; display: flex; align-items: center;">
                <span style="margin-right: 8px;">{status_icon}</span>
                <div>
                    <div style="font-weight: bold; color: {bar_color};">{span['name']}</div>
                    <div style="font-size: 0.8em; color: #888;">{span['type']}</div>
                </div>
            </div>
            
            <!-- Timeline column -->
            <div style="position: relative; height: 32px; background: #2d2d2d; border-radius: 4px; margin: 4px 0;">
                <!-- Timeline bar -->
                <div style="
                    position: absolute;
                    left: {left_percent:.2f}%;
                    width: {max(width_percent, 0.5):.2f}%;
                    height: 24px;
                    top: 4px;
                    background: {bar_color};
                    border-radius: 2px;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    color: white;
                    font-size: 0.75em;
                    font-weight: bold;
                    min-width: 2px;
                "></div>
                
                <!-- Hover info -->
                <div style="
                    position: absolute;
                    right: 5px;
                    top: 6px;
                    font-size: 0.7em;
                    color: #999;
                ">{span['span_id'][:8]}...</div>
            </div>
            
            <!-- Duration column -->
            <div style="padding: 8px; text-align: right; font-size: 0.9em; color: #fff; font-weight: bold;">
                {span['duration_ms']:.1f}ms
            </div>
        </div>
        """
        
        # Add error details if present
        if span['error']:
            html += f"""
            <div style="background: #2d1b1b; border-left: 4px solid #f44336; padding: 8px; margin: 2px 0 4px 20px; font-size: 0.8em;">
                <strong style="color: #f44336;">Error:</strong> <span style="color: #ffcdd2;">{span['error']}</span>
            </div>
            """
    
    html += "</div>"
    
    display(HTML(html))

def show_trace_summary(exporter, limit: int = 10) -> None:
    """Show a trace summary table like traditional APM tools."""
    
    traces = exporter.query_traces(limit=limit)
    
    html = f"""
    <div style="font-family: 'Monaco', 'Menlo', monospace; background: #1e1e1e; color: #d4d4d4; padding: 20px; border-radius: 8px;">
        <h3 style="margin: 0 0 20px 0; color: #4fc3f7;">📊 Recent Traces</h3>
        
        <!-- Table header -->
        <div style="display: grid; grid-template-columns: 2fr 150px 120px 1fr; gap: 15px; padding: 10px 0; border-bottom: 2px solid #404040; font-weight: bold; color: #888;">
            <div>TRACE NAME</div>
            <div>TIMESTAMP</div>
            <div>DURATION</div>
            <div>TRACE ID</div>
        </div>
    """
    
    for i, trace in enumerate(traces):
        trace_id = trace.get('trace_id', 'N/A')
        name = trace.get('name', 'Unnamed')
        timestamp = trace.get('timestamp', 0)
        
        # Format timestamp
        if isinstance(timestamp, (int, float)):
            time_str = datetime.fromtimestamp(timestamp).strftime('%H:%M:%S')
        else:
            time_str = 'N/A'
        
        # Get span count for this trace
        spans = exporter.query_spans(trace_id=trace_id, limit=1000)
        span_count = len(spans)
        
        # Calculate rough duration from spans
        duration_str = "N/A"
        if spans:
            try:
                start_times = []
                end_times = []
                for span in spans:
                    span_data = json.loads(span['data'])
                    start_time = span_data.get('started_at')
                    end_time = span_data.get('ended_at')
                    if start_time and end_time:
                        start_times.append(datetime.fromisoformat(start_time.replace('Z', '+00:00')))
                        end_times.append(datetime.fromisoformat(end_time.replace('Z', '+00:00')))
                
                if start_times and end_times:
                    total_duration = (max(end_times) - min(start_times)).total_seconds() * 1000
                    duration_str = f"{total_duration:.1f}ms"
            except:
                pass
        
        bg_color = "#2d2d2d" if i % 2 == 0 else "#1e1e1e"
        
        html += f"""
        <div style="display: grid; grid-template-columns: 2fr 150px 120px 1fr; gap: 15px; padding: 10px 0; border-bottom: 1px solid #333; background: {bg_color};" 
             onmouseover="this.style.background='#3d3d3d'" onmouseout="this.style.background='{bg_color}'">
            <div>
                <div style="font-weight: bold; color: #4caf50;">{name}</div>
                <div style="font-size: 0.8em; color: #888;">{span_count} spans</div>
            </div>
            <div style="color: #999;">{time_str}</div>
            <div style="color: #fff; font-weight: bold;">{duration_str}</div>
            <div style="font-family: monospace; font-size: 0.8em; color: #888;">{trace_id[:32]}...</div>
        </div>
        """
    
    html += "</div>"
    display(HTML(html))

def show_span_details(span_row: Dict) -> None:
    """Show detailed span information in APM style."""
    
    data = json.loads(span_row['data']) if isinstance(span_row['data'], str) else span_row['data']
    
    span_id = data.get('id', 'N/A')
    trace_id = data.get('trace_id', 'N/A')
    name = data.get('span_data', {}).get('name', 'Unnamed')
    span_type = data.get('span_data', {}).get('type', 'unknown')
    error = data.get('error')
    
    # Parse timing
    started = data.get('started_at', '')
    ended = data.get('ended_at', '')
    duration_ms = None
    
    if started and ended:
        try:
            start_dt = datetime.fromisoformat(started.replace('Z', '+00:00'))
            end_dt = datetime.fromisoformat(ended.replace('Z', '+00:00'))
            duration_ms = (end_dt - start_dt).total_seconds() * 1000
        except:
            pass
    
    status_color = "#f44336" if error else "#4caf50"
    status_text = "ERROR" if error else "SUCCESS"
    
    html = f"""
    <div style="font-family: 'Monaco', 'Menlo', monospace; background: #1e1e1e; color: #d4d4d4; padding: 20px; border-radius: 8px; border-left: 4px solid {status_color};">
        <!-- Header -->
        <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 20px;">
            <h3 style="margin: 0; color: {status_color};">{name}</h3>
            <span style="background: {status_color}; color: white; padding: 4px 12px; border-radius: 4px; font-size: 0.9em; font-weight: bold;">{status_text}</span>
        </div>
        
        <!-- Metrics grid -->
        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin-bottom: 20px;">
            <div style="background: #2d2d2d; padding: 12px; border-radius: 4px;">
                <div style="color: #888; font-size: 0.8em;">DURATION</div>
                <div style="color: #fff; font-size: 1.2em; font-weight: bold;">{f'{duration_ms:.1f}ms' if duration_ms else 'N/A'}</div>
            </div>
            <div style="background: #2d2d2d; padding: 12px; border-radius: 4px;">
                <div style="color: #888; font-size: 0.8em;">TYPE</div>
                <div style="color: #4fc3f7; font-size: 1.2em; font-weight: bold;">{span_type.upper()}</div>
            </div>
            <div style="background: #2d2d2d; padding: 12px; border-radius: 4px;">
                <div style="color: #888; font-size: 0.8em;">START TIME</div>
                <div style="color: #fff; font-size: 1em;">{started.split('T')[1][:12] if 'T' in started else started}</div>
            </div>
        </div>
        
        <!-- IDs section -->
        <div style="background: #2d2d2d; padding: 15px; border-radius: 4px; margin-bottom: 15px;">
            <div style="color: #888; font-size: 0.9em; margin-bottom: 8px;">IDENTIFIERS</div>
            <div style="margin-bottom: 5px;"><strong>Span ID:</strong> <code style="background: #1e1e1e; padding: 2px 6px; border-radius: 3px;">{span_id}</code></div>
            <div><strong>Trace ID:</strong> <code style="background: #1e1e1e; padding: 2px 6px; border-radius: 3px;">{trace_id}</code></div>
        </div>
    """
    
    # Error section
    if error:
        html += f"""
        <div style="background: #2d1b1b; border: 1px solid #f44336; padding: 15px; border-radius: 4px; margin-bottom: 15px;">
            <div style="color: #f44336; font-size: 0.9em; margin-bottom: 8px; font-weight: bold;">ERROR DETAILS</div>
            <code style="color: #ffcdd2; background: #1e1e1e; padding: 8px; border-radius: 3px; display: block; white-space: pre-wrap;">{error}</code>
        </div>
        """
    
    # Tools and metadata
    span_data = data.get('span_data', {})
    tools = span_data.get('tools', [])
    handoffs = span_data.get('handoffs', [])
    
    if tools or handoffs:
        html += f"""
        <div style="background: #2d2d2d; padding: 15px; border-radius: 4px;">
            <div style="color: #888; font-size: 0.9em; margin-bottom: 8px;">METADATA</div>
        """
        if tools:
            html += f"<div style='margin-bottom: 5px;'><strong>Tools:</strong> {', '.join(tools) if tools else 'None'}</div>"
        if handoffs:
            html += f"<div><strong>Handoffs:</strong> {', '.join(handoffs) if handoffs else 'None'}</div>"
        html += "</div>"
    
    html += "</div>"
    
    display(HTML(html))

In [ ]:
show_span(spans[0])

In [ ]:
show_trace(traces[0], show_raw=False)

In [ ]:
show_trace_waterfall(sqlite_tracing_exporter, "trace_022f2a0bc2244fc9ba9a173698701af9")

### Agent Visualization

In [20]:
from agents.extensions.visualization import get_main_graph, get_all_edges, get_all_nodes

In [ ]:
n = get_all_nodes(agent).split(';')
e = get_all_edges(agent).split(';')
g = get_main_graph(agent)

non_null_str = lambda x: x != ""
n = list(filter(non_null_str, n))
e = list(filter(non_null_str, e))

print(f"Found {len(n)} nodes and {len(e)} edges.")
print()
for _n in n:
    print(_n)
print()
for _e in e:
    print(_e)

In [ ]:
from agents.extensions.visualization import draw_graph
draw_graph(agent)

In [ ]:
from agents import WebSearchTool

agent2 = Agent(
    name="Joke generator", 
    instructions="Tell funny jokes.",
    tools=[WebSearchTool()]
)
draw_graph(agent2)

In [ ]:
with trace("Court jester sequence"):
    first_result = await Runner.run(agent2, joker_prompt)
    second_result = await Runner.run(agent2, king_prompt.format(joke=first_result.final_output))
        
display(Markdown(first_result.final_output))
display(Markdown(second_result.final_output))

In [25]:
most_recent_trace_id = sqlite_tracing_exporter.query_traces(limit=1)[0]['trace_id']

In [ ]:
show_trace_waterfall(sqlite_tracing_exporter, most_recent_trace_id)

## Custom LLM

[The docs] propose three modes of custom (non-openai) model usage:
1. Globally use an instance of AsyncOpenAI as the LLM client with [`set_default_openai_client`](https://openai.github.io/openai-agents-python/ref/#agents.set_default_openai_client).
    - e.g., [Nvidia NIM](https://developer.nvidia.com/nim)
2. At `Runner.run` time, use a custom model provider for all agents in this run with [`ModelProvider`](https://openai.github.io/openai-agents-python/ref/models/interface/#agents.models.interface.ModelProvider).
3. At `Agent` construction time, use [`Agent.model`](https://openai.github.io/openai-agents-python/ref/agent/#agents.agent.Agent.model).

### Nvidia NIM

In [ ]:
from agents import function_tool

@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."

agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model='litellm/nvidia_nim/meta/llama-4-maverick-17b-128e-instruct',
    tools=[get_weather],
)

result = await Runner.run(agent, "What's the weather in Tokyo? include the temp!")
print(result.final_output)

### `Runner.run` 

In [ ]:
from agents import RunConfig, Model, ModelProvider, OpenAIChatCompletionsModel, AsyncOpenAI
import os

agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    tools=[get_weather],
)

class CustomModelProvider(ModelProvider):
    def get_model(self) -> Model:
        client = AsyncOpenAI(base_url="https://integrate.api.nvidia.com/v1", api_key=os.environ['NVIDIA_NIM_API_KEY'])
        return OpenAIChatCompletionsModel(model='litellm/nvidia_nim/meta/llama-4-maverick-17b-128e-instruct', openai_client=client)

result = await Runner.run(
    agent, 
    "What's the weather in Beirut? include the temp! Say something that confuses Beirut with Lima.",
    run_config=RunConfig()
)
print(result.final_output)

### `Agent.model` 

In [ ]:
client = AsyncOpenAI(base_url="https://integrate.api.nvidia.com/v1/chat/completions", api_key=os.environ['NVIDIA_NIM_API_KEY'])

agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    tools=[get_weather],
    # model="litellm/anthropic/claude-3-5-sonnet-20240620",
    model='litellm/nvidia_nim/meta/llama-4-maverick-17b-128e-instruct'
)

result = await Runner.run(
    agent,
    "What's the weather in Sao Paulo? Include the forecast for tomorrow. Tell about the the most historic night in Sao Paulo.",
)
print(result.final_output)